In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime 


In [36]:
df = pd.read_excel('пп_станции.xlsx')

In [37]:
df

,Станция,Номер линии,Дата,2024-04-03 00:00:00,2024-04-02 00:00:00,2024-04-01 00:00:00,2024-03-31 00:00:00,2024-03-30 00:00:00,2024-03-29 00:00:00,2024-03-28 00:00:00,...,2024-01-10 00:00:00,2024-01-09 00:00:00,2024-01-08 00:00:00,2024-01-07 00:00:00,2024-01-06 00:00:00,2024-01-05 00:00:00,2024-01-04 00:00:00,2024-01-03 00:00:00,2024-01-02 00:00:00,2024-01-01 00:00:00
0,Б.Рокоссовского,8,СОКОЛЬНИЧЕСКАЯ,16671,15113,2905,13998,12742,1217.0,17624.0,...,7280,4669,3812,5816,15740,13714,2561,11209,2722,4835
1,Черкизовская,8,СОКОЛЬНИЧЕСКАЯ,6248,5441,7991,14864,10015,9721.0,6623.0,...,13486,5061,3663,11353,6924,6496,15991,7029,3999,15638
2,Преображенск. пл,8,СОКОЛЬНИЧЕСКАЯ,14094,1132,6556,7265,4001,10961.0,6852.0,...,8176,9745,13281,15090,13309,8365,2811,5004,17068,17117
3,Сокольники СЛ,8,СОКОЛЬНИЧЕСКАЯ,18651,10083,6488,6409,9446,12051.0,15041.0,...,3637,14001,12049,15972,15545,18759,5573,15017,14176,5465
4,Красносельская,8,СОКОЛЬНИЧЕСКАЯ,7391,19339,15424,10725,5965,14682.0,18134.0,...,14194,7615,9321,16152,4581,7746,8936,11373,19561,1727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,Аэропорт Внуково,1691,СОЛНЦЕВСКАЯ ЛИН.,15787,10257,13493,8656,5761,18352.0,6449.0,...,9654,17815,12401,15277,13153,19801,17569,13950,7545,14178
318,Яхромская,18,ЛЮБЛИНСКАЯ,9813,19663,12491,9180,3563,19023.0,5655.0,...,12471,17980,2322,6341,16395,14759,14798,17189,3864,12536
319,Лианозово,18,ЛЮБЛИНСКАЯ,18978,8499,2959,12203,9070,3349.0,5543.0,...,17639,14071,17237,16304,8357,5616,16496,8603,16046,5489
320,Физтех,18,ЛЮБЛИНСКАЯ,1476,2893,9897,9793,9822,4534.0,4115.0,...,16336,17256,14147,2938,15037,3834,5997,5933,4969,5404


In [38]:
df.shape

(322, 97)

## Preprocessing

In [155]:
def find_count(days,station):
    tod = datetime.datetime.now()
    d = datetime.timedelta(days=days)
    a = (tod - d).replace(hour=0, minute=0, second=0, microsecond=0)
    return df[df['Станция']==station][a]

In [157]:
find_count(19,'Красносельская')

4    15424
Name: 2024-04-01 00:00:00, dtype: int64

## EDA